Задача:
По открытым данным яндекс радара, 71% визитов совершается со смартфонов. На основе публичных данных сделайте прогноз , какая доля визитов в России будет через год и три года 

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.functions import split, trim, col, lower, current_timestamp, left, substring, to_timestamp, regexp_replace, year, sum

import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Указываем путь к JSON-ключу
json_key_file = "9.json"

# Устанавливаем параметры для доступа к Google Sheets API
scope = ['https://www.googleapis.com/auth/spreadsheets']
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_key_file, scope)
gc = gspread.authorize(credentials)

In [0]:
# Указываем ссылку на таблицу Google Sheets
spreadsheet_url = "https://docs.google.com/spreadsheets/d/"

# Открываем таблицу и выбираем лист по имени
worksheet = gc.open_by_url(spreadsheet_url).worksheet("Visits")
# worksheet = gc.open_by_url(spreadsheet_url).sheet1

# Получаем данные из таблицы
data = worksheet.get_all_values()

# Преобразуем данные в Spark DataFrame
columns = data[0]
data = data[1:]

df = spark.createDataFrame(data, columns)

df = df.withColumn("Period", to_timestamp(substring(df["Period"], 1, 10)))

df_visits = df.select(
    col("Period"),
    col("TabletVisits").cast("long"),
    col("DesktopVisits").cast("long"),
    col("SmartphoneVisits").cast("long")
)

df_visits.show()

+-------------------+------------+-------------+----------------+
|             Period|TabletVisits|DesktopVisits|SmartphoneVisits|
+-------------------+------------+-------------+----------------+
|2015-04-01 00:00:00|  1305285165|  12120556907|      2659671927|
|2015-05-01 00:00:00|  1251272550|  11178048024|      2639817070|
|2015-06-01 00:00:00|  1153067654|  10705441080|      2541019098|
|2015-07-01 00:00:00|  1174184403|  10622758998|      2663429598|
|2015-08-01 00:00:00|  1188107395|  10452520053|      2782190607|
|2015-09-01 00:00:00|  1238504001|  11059209494|      3015979125|
|2015-10-01 00:00:00|  1384247601|  12324152809|      3404638098|
|2015-11-01 00:00:00|  1416698186|  12463872897|      3519893361|
|2015-12-01 00:00:00|  1401161167|  12774081709|      3671117648|
|2016-01-01 00:00:00|  1457156089|  12357613779|      3839999700|
|2016-02-01 00:00:00|  1380662398|  12434859740|      3850454785|
|2016-03-01 00:00:00|  1424670668|  12998490161|      4125374902|
|2016-04-0

In [0]:
# Указываем ссылку на таблицу Google Sheets
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1Wef5Gc_UNXmkabDllnsKTUdtcnZYj2nDQejn06FCVbk/edit#gid=0"

# Открываем таблицу и выбираем лист по имени
worksheet = gc.open_by_url(spreadsheet_url).worksheet("Mobile OS Usages")

# Получаем данные из таблицы
data = worksheet.get_all_values()

# Преобразуем данные в Spark DataFrame
columns = data[0]
data = data[1:]

df = spark.createDataFrame(data, columns)
# df = spark.read.format("com.databricks.spark.csv").option("header", "true").load(spreadsheet_url)

df = df.withColumn("Period", to_timestamp(substring(df["Period"], 1, 10)))

df_os_usages = df.select(
    col("Period"),
    col("IOSTotal").cast("long"),
    col("AndroidTotal").cast("long"),
    col("WindowsTotal").cast("long")
)

df_os_usages.show()

+-------------------+----------+------------+------------+
|             Period|  IOSTotal|AndroidTotal|WindowsTotal|
+-------------------+----------+------------+------------+
|2015-04-01 00:00:00| 746091831|  1701638754|   108487178|
|2015-05-01 00:00:00| 735296349|  1694042008|   112833517|
|2015-06-01 00:00:00| 715635422|  1624232111|   109564707|
|2015-07-01 00:00:00| 737038293|  1716835189|   117333272|
|2015-08-01 00:00:00| 762962895|  1805398712|   123493667|
|2015-09-01 00:00:00| 830315295|  1961375556|   136120626|
|2015-10-01 00:00:00| 926348543|  2233272524|   152020302|
|2015-11-01 00:00:00| 970402652|  2308132299|   154195438|
|2015-12-01 00:00:00| 996704799|  2429648475|   159761874|
|2016-01-01 00:00:00|1026850230|  2567480828|   164219581|
|2016-02-01 00:00:00|1041862763|  2571018293|   161369199|
|2016-03-01 00:00:00|1126254401|  2751819664|   169130258|
|2016-04-01 00:00:00|1091659942|  2634180751|   159101999|
|2016-05-01 00:00:00|1091611646|  2607526782|   15443716

In [0]:
merged_data = df_visits.join(df_os_usages, "Period", "inner")

display(merged_data)

Period,TabletVisits,DesktopVisits,SmartphoneVisits,IOSTotal,AndroidTotal,WindowsTotal
2015-04-01T00:00:00Z,1305285165,12120556907,2659671927,746091831,1701638754,108487178
2015-05-01T00:00:00Z,1251272550,11178048024,2639817070,735296349,1694042008,112833517
2015-06-01T00:00:00Z,1153067654,10705441080,2541019098,715635422,1624232111,109564707
2015-07-01T00:00:00Z,1174184403,10622758998,2663429598,737038293,1716835189,117333272
2015-08-01T00:00:00Z,1188107395,10452520053,2782190607,762962895,1805398712,123493667
2015-09-01T00:00:00Z,1238504001,11059209494,3015979125,830315295,1961375556,136120626
2015-10-01T00:00:00Z,1384247601,12324152809,3404638098,926348543,2233272524,152020302
2015-11-01T00:00:00Z,1416698186,12463872897,3519893361,970402652,2308132299,154195438
2015-12-01T00:00:00Z,1401161167,12774081709,3671117648,996704799,2429648475,159761874
2016-01-01T00:00:00Z,1457156089,12357613779,3839999700,1026850230,2567480828,164219581


In [0]:

merged_data = merged_data.withColumn("Year", year("Period"))

merged_data = merged_data.groupBy("Year").agg(
    sum("TabletVisits").alias("TabletVisits"),
    sum("DesktopVisits").alias("DesktopVisits"),
    sum("SmartphoneVisits").alias("SmartphoneVisits"),
    sum("IOSTotal").alias("IOSTotal"),
    sum("AndroidTotal").alias("AndroidTotal"),
    sum("WindowsTotal").alias("WindowsTotal")
)

display(merged_data)

Year,TabletVisits,DesktopVisits,SmartphoneVisits,IOSTotal,AndroidTotal,WindowsTotal
2015,11512528122,103700641971,26897756532,7420796079,17474575628,1173810581
2016,15244481075,137921551093,49971216426,13743814063,33602566677,1854086063
2017,13730054174,134599460500,69765970222,19203242897,48630348010,1500406802
2018,11568043310,122676874972,97172562820,25018395394,70918708974,989952095
2019,9327617398,110317427938,124041057976,28534214626,94875538913,493153695
2020,8724875099,114551855431,152596999767,32977912193,119355812949,186060096
2021,7078915005,94186261631,164436829995,37742652533,126491494054,62248391
2023,5629144007,91010013529,203477292839,41835908740,161599811316,14506385
2022,6066525332,90328235354,174779148054,40559160196,133839992057,26600294
2024,1382808963,24199802426,62110637054,11234117878,50867342442,2644268


In [0]:
merged_data = merged_data.withColumn("SmartphoneShare",col("SmartphoneVisits") / (col("SmartphoneVisits") + col("TabletVisits") + col("DesktopVisits")))
merged_data = merged_data.withColumn("DesktopShare",col("DesktopVisits") / (col("SmartphoneVisits") + col("TabletVisits") + col("DesktopVisits")))
merged_data = merged_data.withColumn("TabletShare",col("TabletVisits") / (col("SmartphoneVisits") + col("TabletVisits") + col("DesktopVisits")))

merged_data = merged_data.withColumn("IOSShare",col("IOSTotal") / (col("IOSTotal") + col("AndroidTotal") + col("WindowsTotal")))
merged_data = merged_data.withColumn("AndroidShare",col("AndroidTotal") / (col("IOSTotal") + col("AndroidTotal") + col("WindowsTotal")))
merged_data = merged_data.withColumn("WindowsShare",col("WindowsTotal") / (col("IOSTotal") + col("AndroidTotal") + col("WindowsTotal")))


display(merged_data)


Year,TabletVisits,DesktopVisits,SmartphoneVisits,IOSTotal,AndroidTotal,WindowsTotal,SmartphoneShare,DesktopShare,TabletShare,IOSShare,AndroidShare,WindowsShare
2015,11512528122,103700641971,26897756532,7420796079,17474575628,1173810581,0.18927296563885873,0.7297161761856888,0.08101085817545242,0.2846578000421553,0.670315448906266,0.04502675105157867
2016,15244481075,137921551093,49971216426,13743814063,33602566677,1854086063,0.24599730857768437,0.67895746372275,0.07504522769956563,0.2793431639181515,0.6829725175483717,0.03768431853347674
2017,13730054174,134599460500,69765970222,19203242897,48630348010,1500406802,0.31988727439849696,0.6171583999741419,0.06295432562736111,0.27696719548175275,0.7013925291615987,0.021640275356648555
2018,11568043310,122676874972,97172562820,25018395394,70918708974,989952095,0.4199015664558636,0.5301106657449474,0.049987767799189065,0.25811570377720366,0.7316709241146906,0.010213372108105797
2019,9327617398,110317427938,124041057976,28534214626,94875538913,493153695,0.5090198262852347,0.45270299142481946,0.038277182289945845,0.2302949564541773,0.7657248811266419,0.0039801624191807055
2020,8724875099,114551855431,152596999767,32977912193,119355812949,186060096,0.5531407416092761,0.4152329230756253,0.031626335315098605,0.2162205522486116,0.782559539818069,0.001219907933319352
2021,7078915005,94186261631,164436829995,37742652533,126491494054,62248391,0.6188768842207714,0.35448080661958803,0.02664230915964068,0.22972295002610316,0.7698981713563329,3.788786175639175E-4
2023,5629144007,91010013529,203477292839,41835908740,161599811316,14506385,0.6779944670968622,0.3032490002306825,0.01875653267245531,0.20563215618800157,0.7942965419252728,7.130188672563022E-5
2022,6066525332,90328235354,174779148054,40559160196,133839992057,26600294,0.6445278930635515,0.3331007609607685,0.022371345975679947,0.23252965576325152,0.7673178421342116,1.5250210253690838E-4
2024,1382808963,24199802426,62110637054,11234117878,50867342442,2644268,0.708271596237782,0.2759596987871843,0.015768704975033694,0.1808917132374323,0.8190657087716677,4.2577990899991755E-5


In [0]:
# Указываем ссылку на таблицу Google Sheets
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1Wef5Gc_UNXmkabDllnsKTUdtcnZYj2nDQejn06FCVbk/edit#gid=0"

# Открываем таблицу и выбираем лист по имени
worksheet = gc.open_by_url(spreadsheet_url).worksheet("Smartphone Users")

# Получаем данные из таблицы
data = worksheet.get_all_values()

# Преобразуем данные в Spark DataFrame
columns = data[0]
data = data[1:]

df = spark.createDataFrame(data, columns)

df_sm_users = df.select(
    col("Year").alias("Year_1"),
    col("SmartphoneUsers").cast("long")
)

df_sm_users.show()

+------+---------------+
|Year_1|SmartphoneUsers|
+------+---------------+
|  2014|       47310000|
|  2015|       47940000|
|  2016|       53900000|
|  2017|       59180000|
|  2018|       68790000|
|  2019|       73500000|
|  2020|       78470000|
|  2021|       83550000|
|  2022|       90230000|
|  2023|       97470000|
|  2024|      104310000|
|  2025|      110910000|
|  2026|      117170000|
|  2027|      121680000|
|  2028|      121710000|
|  2029|      121710000|
+------+---------------+



In [0]:
# Указываем ссылку на таблицу Google Sheets
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1Wef5Gc_UNXmkabDllnsKTUdtcnZYj2nDQejn06FCVbk/edit#gid=0"

# Открываем таблицу и выбираем лист по имени
worksheet = gc.open_by_url(spreadsheet_url).worksheet("Mobile internet users ")

# Получаем данные из таблицы
data = worksheet.get_all_values()

# Преобразуем данные в Spark DataFrame
columns = data[0]
data = data[1:]

df = spark.createDataFrame(data, columns)

df_internet_users = df.select(
    col("Year").alias("Year_1"),
    col("InternetUsers").cast("long")
)

df_internet_users.show()

+------+-------------+
|Year_1|InternetUsers|
+------+-------------+
|  2010|     20000000|
|  2011|     30000000|
|  2012|     39000000|
|  2013|     42000000|
|  2014|     44000000|
|  2015|     44500000|
|  2016|     52000000|
|  2017|     59000000|
|  2018|     67000000|
|  2019|     70000000|
|  2020|     78000000|
|  2021|     82000000|
|  2022|     90000000|
|  2023|     98000000|
|  2024|    103000000|
|  2025|    110000000|
|  2026|    118000000|
|  2027|    121000000|
|  2028|    121000000|
|  2029|    121000000|
+------+-------------+



In [0]:
# Указываем ссылку на таблицу Google Sheets
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1Wef5Gc_UNXmkabDllnsKTUdtcnZYj2nDQejn06FCVbk/edit#gid=0"

# Открываем таблицу и выбираем лист по имени
worksheet = gc.open_by_url(spreadsheet_url).worksheet("Device Penetration Rate")

# Получаем данные из таблицы
data = worksheet.get_all_values()

# Преобразуем данные в Spark DataFrame
columns = data[0]
data = data[1:]

df = spark.createDataFrame(data, columns)

df = df.withColumn("Penetration", regexp_replace("Penetration", ",", "."))

df_penetration = df.select(
    col("Year").alias("Year_1"),
    col("Penetration").cast("double")
)

df_penetration.show()

+------+-----------+
|Year_1|Penetration|
+------+-----------+
|  2014|       0.45|
|  2015|       0.47|
|  2016|        0.5|
|  2017|       0.55|
|  2018|       0.64|
|  2019|       0.66|
|  2020|       0.69|
|  2021|        0.7|
|  2022|       0.74|
|  2023|        0.8|
|  2024|       0.82|
|  2025|       NULL|
|  2026|       NULL|
|  2027|       NULL|
|  2028|       NULL|
|  2029|       0.88|
+------+-----------+



In [0]:
df_final = merged_data\
.join(
    df_sm_users, merged_data["Year"] == df_sm_users["Year_1"], "left")\
.join(
    df_internet_users, merged_data["Year"] == df_internet_users["Year_1"], "left")\
.join(
    df_penetration, merged_data["Year"] == df_penetration["Year_1"], "left")\
.drop("Year_1")

display(df_final)

Year,TabletVisits,DesktopVisits,SmartphoneVisits,IOSTotal,AndroidTotal,WindowsTotal,SmartphoneShare,DesktopShare,TabletShare,IOSShare,AndroidShare,WindowsShare,SmartphoneUsers,InternetUsers,Penetration
2015,11512528122,103700641971,26897756532,7420796079,17474575628,1173810581,0.18927296563885873,0.7297161761856888,0.08101085817545242,0.2846578000421553,0.670315448906266,0.04502675105157867,47940000,44500000,0.47
2016,15244481075,137921551093,49971216426,13743814063,33602566677,1854086063,0.24599730857768437,0.67895746372275,0.07504522769956563,0.2793431639181515,0.6829725175483717,0.03768431853347674,53900000,52000000,0.5
2017,13730054174,134599460500,69765970222,19203242897,48630348010,1500406802,0.31988727439849696,0.6171583999741419,0.06295432562736111,0.27696719548175275,0.7013925291615987,0.021640275356648555,59180000,59000000,0.55
2018,11568043310,122676874972,97172562820,25018395394,70918708974,989952095,0.4199015664558636,0.5301106657449474,0.049987767799189065,0.25811570377720366,0.7316709241146906,0.010213372108105797,68790000,67000000,0.64
2019,9327617398,110317427938,124041057976,28534214626,94875538913,493153695,0.5090198262852347,0.45270299142481946,0.038277182289945845,0.2302949564541773,0.7657248811266419,0.0039801624191807055,73500000,70000000,0.66
2020,8724875099,114551855431,152596999767,32977912193,119355812949,186060096,0.5531407416092761,0.4152329230756253,0.031626335315098605,0.2162205522486116,0.782559539818069,0.001219907933319352,78470000,78000000,0.69
2021,7078915005,94186261631,164436829995,37742652533,126491494054,62248391,0.6188768842207714,0.35448080661958803,0.02664230915964068,0.22972295002610316,0.7698981713563329,3.788786175639175E-4,83550000,82000000,0.7
2023,5629144007,91010013529,203477292839,41835908740,161599811316,14506385,0.6779944670968622,0.3032490002306825,0.01875653267245531,0.20563215618800157,0.7942965419252728,7.130188672563022E-5,97470000,98000000,0.8
2022,6066525332,90328235354,174779148054,40559160196,133839992057,26600294,0.6445278930635515,0.3331007609607685,0.022371345975679947,0.23252965576325152,0.7673178421342116,1.5250210253690838E-4,90230000,90000000,0.74
2024,1382808963,24199802426,62110637054,11234117878,50867342442,2644268,0.708271596237782,0.2759596987871843,0.015768704975033694,0.1808917132374323,0.8190657087716677,4.2577990899991755E-5,104310000,103000000,0.82


In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

# Определение признаков и целевой переменной
features = ['TabletVisits', 'DesktopVisits', 'SmartphoneVisits', 'IOSTotal', 'AndroidTotal', 'WindowsTotal', 
            'DesktopShare', 'TabletShare', 'IOSShare', 'AndroidShare', 
            'WindowsShare', 'SmartphoneUsers', 'InternetUsers', 'Penetration']

# features = ['SmartphoneUsers', 'InternetUsers'] 

target = 'SmartphoneShare'

model = LinearRegression()

df = df_final.toPandas()

# Обучение модели на данных
model.fit(df[features].values, df[target].values)

# Вывод коэффициентов модели
print("Coefficients:", model.coef_)
print("Intercept:", model.intercept_)

# Оценка производительности модели с использованием перекрестной проверки
scores = cross_val_score(model, df[features], df[target], cv=5)  # cv - количество разбиений (фолдов) в перекрестной проверке

# Вывод средней оценки производительности модели
print("Average model score:", scores.mean())




Coefficients: [-5.54895380e-13  4.14162890e-13 -3.35057016e-11  3.67934347e-11
  3.26641274e-11  6.68562960e-12 -6.38919574e-01 -9.13324809e-02
 -1.54639753e-01  2.11773320e-01 -5.71335668e-02  8.37670145e-09
 -2.44805232e-09 -4.91595945e-01]
Intercept: 0.5189397058741405
Average model score: -22.12815437343982


In [0]:
predict_df = df_sm_users.join(df_internet_users, "Year_1", "inner").withColumn("Year", col("Year_1")).drop("Year_1")
predict_df = predict_df.filter(predict_df.Year >= 2025)
display(predict_df)

SmartphoneUsers,InternetUsers,Year
110910000,110000000,2025
117170000,118000000,2026
121680000,121000000,2027
121710000,121000000,2028
121710000,121000000,2029


In [0]:
# скрыл ячейку с прогнозом линейной регресии, тк недостаточно данных по фичам
# import pandas as pd
# import numpy as np
# from sklearn.linear_model import LinearRegression

# # Ваш DataFrame с данными
# new_data = predict_df.toPandas()

# # Определение признаков и целевой переменной
# features = ['SmartphoneUsers', 'InternetUsers']
# target = 'SmartphoneShare'

# # Получение прогноза на новые года
# new_predictions = model.predict(new_data[features])

# # Добавление прогнозов к DataFrame для новых годов
# new_data['SmartphoneShare_Predictions'] = new_predictions

# # Вывод DataFrame с прогнозами
# print(new_data)


In [0]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# Преобразование DataFrame во временной ряд

df = df_final.toPandas()

time_series = df.set_index('Year')['SmartphoneShare']

# Обучение модели ARIMA
model = ARIMA(time_series, order=(1, 2, 0))
model_fit = model.fit()

# Прогнозирование на несколько будущих лет
future_years = [2025, 2026, 2027]
forecast = model_fit.forecast(steps=len(future_years))

# Создание DataFrame с прогнозами
forecast_df = pd.DataFrame({'Year': future_years, 'SmartphoneShare_Predictions': forecast})

# Вывод DataFrame с прогнозами
print(forecast_df)


    Year  SmartphoneShare_Predictions
10  2025                     0.714146
11  2026                     0.754470
12  2027                     0.774286


/databricks/python/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/databricks/python/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/databricks/python/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/databricks/python/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
